In [2]:
import os
import shutil
import subprocess as sp

root_dir = '/home/lonya/doc/deeph/demo-CrI3'
os.chdir(root_dir)

i_max, j_max = 20, 5
train_i, train_j = 100, 10

## 结构处理

In [3]:
import dpdata
import numpy as np
from pymatgen.core.structure import Structure

os.chdir(root_dir)

out_dir = "./dataset"
kpt_file = fr"""K_POINTS
0
Gamma
3 3 3 0 0 0
"""
input_file = fr"""INPUT_PARAMETERS
ntype                   2
noncolin                1
lspinorb                0 
calculation             scf
basis_type              lcao
ecutwfc                 100
scf_thr                 1.0e-4
scf_nmax                100
gamma_only              0

ks_solver               genelpa
smearing_method         gaussian
smearing_sigma          0.05

mixing_type             pulay
mixing_beta             0.4

out_mat_hs2             1
"""

def from_data_to_dataset(input_dir, output_dir, kpt, input):
    lat = np.loadtxt(os.path.join(input_dir, "lat.dat"))
    elements = np.loadtxt(os.path.join(input_dir, "element.dat"))
    site_positions = np.loadtxt(os.path.join(input_dir, "site_positions.dat"))
    magmom = np.loadtxt(os.path.join(input_dir, "magmom.txt"))

    #print(lat)
    #print(np.shape(site_positions))
    #print(elements)
    #print(len(elements))
    #print(len(site_positions.T))
    
    os.makedirs(output_dir, exist_ok=True)
    Structure(lat,elements,site_positions.T,coords_are_cartesian=True,to_unit_cell=True).to(os.path.join(output_dir, f'CrI3.vasp'), 'POSCAR')
    dpdata.System(os.path.join(output_dir, 'CrI3.vasp'),fmt="vasp/poscar",).to("abacus/stru", os.path.join(output_dir, "STRU"), mass=[51.996, 126.905], pp_file=["/home/lonya/doc/abacus/pseudo/Cr_ONCV_PBE-1.0.upf", "/home/lonya/doc/abacus/pseudo/I_ONCV_PBE-1.0.upf"], numerical_orbital=["/home/lonya/doc/abacus/orbital/24_Cr_DZP/Cr_gga_7au_100Ry_4s2p2d1f.orb", "/home/lonya/doc/abacus/orbital/53_I_DZP/I_gga_6au_100Ry_2s2p2d1f.orb"])

    # change spin moment
    if True:
        file_path = os.path.join(output_dir, "STRU")
    with open(file_path, 'r') as file:
        lines = file.readlines()
    for k in range(len(lines)):
        if lines[k].strip() == 'Cr' and k+2 < len(lines) and lines[k+1].strip() == '0.0':
            atoms = int(lines[k+2])
            for atom in range(atoms):
                lines[k+3+atom] = lines[k+3+atom].strip() + " mag " + str(magmom[atom][1]) + " angle1 " + str(magmom[atom][2]) + " angle2 " + str(magmom[atom][2]) + "\n"
        if lines[k].strip() == 'I' and k+2 < len(lines) and lines[k+1].strip() == '0.0':
            atoms = int(lines[k+2])
            for atom in range(atoms):
                lines[k+3+atom] = lines[k+3+atom].strip() + " mag " + str(magmom[8+atom][1]) + " angle1 " + str(magmom[8+atom][2]) + " angle2 " + str(magmom[8+atom][2]) + "\n"
    with open(file_path, 'w') as f:
        f.writelines(lines)
    with open(os.path.join(output_dir, "KPT"), 'w') as f:
        f.write(kpt)
    with open(os.path.join(output_dir, "INPUT"), 'w') as f:
        f.write(input)

for i in range(1, i_max + 1):
    jj = 0
    for j in range(1, j_max + 1):
        dir = f"data/{i}_{j + jj}"
        while os.path.exists(dir) == False:
            jj += 1
            dir = f"data/{i}_{j + jj}"
        from_data_to_dataset(dir, os.path.join(out_dir, str(i) + "_" + str(j)), kpt_file, input_file)


In [33]:
os.environ['OMP_NUM_THREADS'] = '8'
import time
'''
i_last = 4
j_last = 5
'''
start_time = time.time()
for i in range(1, i_max + 1):
    if i < i_last:
        continue
    i_last = 0
    for j in range(1, j_max + 1):
        if j < j_last:
            continue
        j_last = 0
        work_dir = os.path.join(root_dir, "dataset", str(i) + "_" + str(j))
        os.chdir(work_dir)
        #sp.run(['mpirun', '-np', '1', 'abacus'])
        work_start_time = time.time()
        print(f'{i}_{j} start')
        sp.run(['mpirun', '-np', '1', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
        print(f'{i}_{j} done, used: {time.time() - work_start_time}, total: {time.time() - start_time}')



4_5 start
4_5 done, used: 1035.70871758461, total: 1035.7089896202087
5_1 start
5_1 done, used: 968.1318635940552, total: 2003.8410873413086
5_2 start
5_2 done, used: 967.9457330703735, total: 2971.7871012687683
5_3 start
5_3 done, used: 967.2639305591583, total: 3939.051101207733
5_4 start
5_4 done, used: 969.2385196685791, total: 4908.289867639542
5_5 start
5_5 done, used: 967.9526753425598, total: 5876.242616176605
6_1 start
6_1 done, used: 934.812328338623, total: 6811.055144071579
6_2 start
6_2 done, used: 935.1356375217438, total: 7746.190991640091
6_3 start
6_3 done, used: 935.3992321491241, total: 8681.590296030045
6_4 start
6_4 done, used: 934.7605938911438, total: 9616.351128339767
6_5 start
6_5 done, used: 935.1298015117645, total: 10551.481000423431
7_1 start
7_1 done, used: 935.350977897644, total: 11486.832047700882
7_2 start
7_2 done, used: 935.9198105335236, total: 12422.751928806305
7_3 start
7_3 done, used: 935.6832659244537, total: 13358.43539595604
7_4 start
7_4 don

In [4]:
os.environ['OMP_NUM_THREADS'] = '8'
import time
start_time = time.time()
targets = ["1_1"]
for target in targets:
    work_dir = os.path.join(root_dir, "dataset", target)
    os.chdir(work_dir)
   
    work_start_time = time.time()
    print(f'{target} start')
    sp.run(['mpirun', '-np', '1', 'abacus'])
    #sp.run(['mpirun', '-np', '1', 'abacus'], stdout=sp.DEVNULL, stderr=sp.DEVNULL)
    print(f'{target} done, used: {time.time() - work_start_time}, total: {time.time() - start_time}')

1_1 start
                                                                                     
                              ABACUS v3.6.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 059fc16 (Sat Apr 13 12:08:33 2024 +0800)

 Sun May 19 16:29:47 2024
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : CPU / 13th Gen Intel(R) Core(TM) i9-13900K

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%% WARNING  WARNING  WARNING  WARNING  WARNING  %%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
!!! WARNING: Some atoms are too close!!!
!!! Plea

 GE71   -8.292853e+04  -7.145209e-05  1.485e-04  2.381e+02  
 GE72   -8.292851e+04  1.775051e-02   1.566e-04  2.375e+02  
 GE73   -8.292844e+04  7.352213e-02   1.838e-04  2.375e+02  
 GE74   -8.292849e+04  -5.561903e-02  3.928e-04  2.377e+02  
 GE75   -8.292850e+04  -1.118686e-02  2.779e-04  2.372e+02  
 GE76   -8.292850e+04  -9.113921e-04  3.513e-04  2.374e+02  
 GE77   -8.292853e+04  -2.482732e-02  2.246e-04  2.380e+02  
 GE78   -8.292822e+04  3.048670e-01   2.368e-04  2.376e+02  
 GE79   -8.292845e+04  -2.290115e-01  8.260e-04  2.370e+02  
 GE80   -8.292853e+04  -7.392327e-02  3.979e-04  2.378e+02  
[mpiexec@lonya-ubuntu] Sending Ctrl-C to processes as requested
[mpiexec@lonya-ubuntu] Press Ctrl-C again to force abort

=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES
=   RANK 0 PID 25039 RUNNING AT lonya-ubuntu
=   KILLED BY SIGNAL: 2 (Interrupt)


KeyboardInterrupt: 

In [73]:
for i in range(1, train_i + 1):
    for j in range(1, train_j + 1):
        shutil.copytree(os.path.join(root_dir, "dataset", str((i - 1) % 20 + 1) + "_" + str((j - 1) % 5 + 1)), os.path.join(root_dir, "raw", str(i) + "_" + str(j)), dirs_exist_ok=True)

In [55]:
# magmom for abacus STRU
# changed in preprocess.py
def collect_magmom_from_abacus(input_dir, output_dir, num_atom, mag_element):
    magmom_data = np.zeros((num_atom, 4))
    index_atom = 0

    with open(os.path.join(input_dir, "STRU"), 'r') as file:
        lines = file.readlines()
        for k in range(len(lines)):
            if lines[k].strip() == 'ATOMIC_POSITIONS':
                kk = k + 2
                while kk < len(lines):
                    if lines[kk] == "\n":
                        continue
                    element_str = lines[kk].strip()
                    element_amount = int(lines[kk + 2].strip())
                    for j in range(element_amount):
                        line = lines[kk + 3 + j].strip().split()
                        if len(line) < 11: # check if magmom is included
                            raise ValueError('this line do not contain magmom: {} in this file: {}'.format(line, input_dir))
                        if line[7] != "angle1" and line[8] != "angle1": # check if magmom is in angle mode
                            raise ValueError('mag in STRU should be mag * angle1 * angle2 *')
                        if line[6] == "mag": # for if 'm' is included
                            index_str = 7
                        else:
                            index_str = 8
                        magmom_data[index_atom] = int(element_str in mag_element), line[index_str], line[index_str + 2], line[index_str + 4]
                        index_atom += 1
                    kk += 3 + element_amount

    np.savetxt(os.path.join(output_dir, "magmom.txt"), magmom_data)


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (405693214.py, line 23)

In [6]:
os.chdir(root_dir)
sp.run(["python", "preprocess_modified.py", "--config", "preprocess.ini"])

User config name: ['preprocess.ini']
Found 1 directories to preprocess
Preprocessing No. 1/1 [08:00:00<?]...Output subdirectories: OUT.ABACUS

Preprocess finished in 1.15 seconds


CompletedProcess(args=['python', 'preprocess_modified.py', '--config', 'preprocess.ini'], returncode=0)

In [4]:
import json
for i in range(1, 101):
    # change isspinful
    dir = os.path.join(root_dir, "processed", str(i) + "_1/info.json")
    with open(dir, 'r') as file:
        data = json.load(file)
        data['isspinful'] = True
    with open(dir, 'w') as file:
        json.dump(data, file, indent=4)

    # copy magmom
    for j in range(1, 11):
        dir = os.path.join(root_dir, "data", str(i) + "_" + str(j), "magmom.txt")
        if os.path.exists(dir):
            break
    shutil.copyfile(dir, os.path.join(root_dir, "processed", str(i) + "_1/magmom.txt"))

    # clone to 10
    for j in range(2, 11):
        shutil.copytree(os.path.join(root_dir, "processed", str(i) + "_1"), os.path.join(root_dir, "processed", str(i) + "_" + str(j)), dirs_exist_ok=True)

In [8]:
sp.run(['python', 'xDeepH/xdeeph-train-debuged.py', 'CrI3_monolayer_redo.ini', '-n', '8'])

Loading train config from: CrI3_monolayer_redo.ini

------- xDeepH model training begins -------
Output will be stored under: ./results_redo/monolayer_CrI3/2024-05-17_10-40-11
Using random seed: 42
Data type during training: torch.float32
Saved xDeepH source code to output dir

------- Preparation of training data -------

Processing graph data...
Graph data file: HGraph-CrI3_monolayer_RM_100x10-rFromDFT-edge=Aij.pkl
Process new data file......
Looking for preprocessed data under: ./processed
Found 1000 structures, have cost 0 seconds
  0%|          | 0/1000 [00:00<?, ?it/s]/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph/graph.py:146: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  Aij_dict[key] = torch.tensor(v, dtype=flt2cplx(default_dtype_torch))
Traceback (most recent call la

CompletedProcess(args=['python', 'xDeepH/xdeeph-train-debuged.py', 'CrI3_monolayer_redo.ini', '-n', '8'], returncode=1)

In [52]:
os.chdir(root_dir)
sp.run(['python', 'xDeepH/xdeeph-eval.py', 'eval_default.ini'])

Loading evaluation config from: eval_default.ini

------- Preparation of graph data for evaluation -------

Processing graph data...
Graph data file: HGraph-1-rFromDFT-edge=Aij.pkl
Process new data file......
Looking for preprocessed data under: ./processed/1_1
Found 1 structures, have cost 0 seconds
Finish processing 1 structures, have cost 0 seconds
Finished saving 1 structures to raw_data_file, have cost 0 seconds
Finish loading the processed 1 structures (spinful: False, the number of atomic types: 2), cost 0.00 seconds

------- Finding trained models -------
Successfully found 2 model(s):
model 0: /home/lonya/doc/deeph/demo-CrI3/results/monolayer_CrI3/2024-05-12_19-59-30
model 1: /home/lonya/doc/deeph/demo-CrI3/results/monolayer_CrI3/2024-05-13_10-03-45

------- Evaluating model -------

Loading model 0:
Loading train config from: /home/lonya/doc/deeph/demo-CrI3/results/monolayer_CrI3/2024-05-12_19-59-30/src/train.ini


100%|██████████| 1/1 [00:00<00:00, 67.08it/s]
Traceback (most recent call last):
  File "/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph-eval.py", line 29, in <module>
    kernel.eval(args.config, debug=args.debug)
  File "/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph/kernel.py", line 351, in eval
    self.config_set_target()
  File "/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph/kernel.py", line 572, in config_set_target
    self.train_config.set_target(o, i, s, verbose)
  File "/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph/parse_configs.py", line 196, in set_target
    target_blocks, net_out_irreps, irreps_post_edge, net_out_tr = orbital_analysis(
                                                                  ^^^^^^^^^^^^^^^^^
  File "/home/lonya/doc/deeph/demo-CrI3/xDeepH/xdeeph/utils.py", line 442, in orbital_analysis
    return targets, net_out_irreps, net_out_irreps.sort()[0].simplify(), net_out_tr
                                                                         ^^^^^^^^^

CompletedProcess(args=['python', 'xDeepH/xdeeph-eval.py', 'eval_default.ini'], returncode=1)

# 能带图

In [7]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import hermtool

os.chdir('/home/lonya/doc/deeph/demo-CrI3/processed/1_1')

In [39]:
with h5py.File('hamiltonians.h5', 'r') as file:
    def print_name(name):
        print(name)
    file.visit(print_name)
    
    '''
    def get_name(name):
        return name
    elem = file.name
    print(elem)
    elements = file.visit(get_name).strip()
    a, max = 4, 0
    print(elements)
    while a < len(elements):
        e = int(elements[a][:-1])
        if e > max:
            max = e
        a += 5
    print(max)
    '''
    #file["-1, -1, 0, 12, 10"].visit(print_name)
print('---------------------')
with h5py.File('overlaps.h5', 'r') as file:
    def print_name(name):
        print(name)
    file.visit(print_name)

[-1, -1, 0, 12, 10]
[-1, -1, 0, 12, 19]
[-1, -1, 0, 12, 1]
[-1, -1, 0, 12, 22]
[-1, -1, 0, 12, 30]
[-1, -1, 0, 12, 6]
[-1, -1, 0, 13, 10]
[-1, -1, 0, 13, 14]
[-1, -1, 0, 13, 19]
[-1, -1, 0, 13, 22]
[-1, -1, 0, 13, 27]
[-1, -1, 0, 13, 2]
[-1, -1, 0, 13, 6]
[-1, -1, 0, 17, 10]
[-1, -1, 0, 17, 6]
[-1, -1, 0, 21, 10]
[-1, -1, 0, 21, 14]
[-1, -1, 0, 21, 19]
[-1, -1, 0, 21, 22]
[-1, -1, 0, 21, 27]
[-1, -1, 0, 21, 2]
[-1, -1, 0, 21, 6]
[-1, -1, 0, 28, 22]
[-1, -1, 0, 29, 10]
[-1, -1, 0, 29, 11]
[-1, -1, 0, 29, 14]
[-1, -1, 0, 29, 19]
[-1, -1, 0, 29, 1]
[-1, -1, 0, 29, 22]
[-1, -1, 0, 29, 27]
[-1, -1, 0, 29, 2]
[-1, -1, 0, 29, 30]
[-1, -1, 0, 29, 6]
[-1, -1, 0, 29, 7]
[-1, -1, 0, 5, 10]
[-1, -1, 0, 5, 14]
[-1, -1, 0, 5, 19]
[-1, -1, 0, 5, 27]
[-1, -1, 0, 5, 6]
[-1, -1, 0, 9, 10]
[-1, -1, 0, 9, 6]
[-1, 0, 0, 12, 1]
[-1, 0, 0, 13, 14]
[-1, 0, 0, 13, 1]
[-1, 0, 0, 13, 22]
[-1, 0, 0, 13, 23]
[-1, 0, 0, 13, 30]
[-1, 0, 0, 13, 6]
[-1, 0, 0, 15, 1]
[-1, 0, 0, 15, 22]
[-1, 0, 0, 16, 10]
[-1, 0, 0, 16,

In [49]:

hermtool.get_matrix_h5('./')


eigenvalues, eigenvectors = np.linalg.eigh(hamiltonian)

print("本征值：", eigenvalues[:10])

plt.figure()
plt.plot(eigenvalues, marker='o')
plt.title('Energy Spectrum')
plt.xlabel('Index')
plt.ylabel('Energy')
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (32,) + inhomogeneous part.